In [22]:

from openai import OpenAI
import mysql.connector
from pydantic import BaseModel, Field
import json

client = OpenAI(api_key='')

In [9]:
indic_base= client.files.create(
    file= open('indic_base.pdf', 'rb'),
    purpose= 'user_data'
)
indic_lic1= client.files.create(
    file= open('indic2.pdf', 'rb'),
    purpose= 'user_data'
)

In [12]:
# Conexión a la base de datos:
class ConexionMySQL:
    def __init__(self, host, port, user, password, database):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database

        # Aquí iría la lógica para conectarse a MySQL
        self.conexion = mysql.connector.connect(
            host=self.host,
            port=self.port,
            user=self.user,
            password=self.password,
            database=self.database
        )
        self.cursor = self.conexion.cursor()
        if self.conexion.is_connected():
            # Verificar la conexión
            print("Conexión exitosa a la base de datos MySQL")
        else:
            print("Error al conectar a la base de datos MySQL")

In [ ]:
def main_mysql():
    # Crear una instancia de la clase ConexionMySQL
    db_mysql = ConexionMySQL(
        host="localhost",
        port=3306,
        user = "root",
        password = "admin",
        database = "db_insitel"
    )

    # Ejecutar una consulta a la base de datos
    db_mysql.cursor.execute("SELECT * FROM indicadores_financieros;")

    # Obtener los resultados
    resultados = db_mysql.cursor.fetchall()
    # Imprimir los resultados
    for fila in resultados:
        print(fila)

    # Cerrar la conexión
    db_mysql.cursor.close()
    db_mysql.conexion.close()
    print("Conexión cerrada")
main_mysql()

In [15]:
# Modelo:
class Indicadores(BaseModel):
    descrip: str = Field(description='Indice Solicitado:')
    valor: float = Field(description='Valor:')
    cumple: bool = Field(description='Cumple / No Cumple:')

In [18]:
# Prompt propuesto:
response = client.responses.create(
    model='gpt-4o-mini',
    input=[
        {
            'role': 'user',
            'content': [
                {'type': 'input_file', 'file_id': indic_base.id},
                {'type': 'input_file', 'file_id': indic_lic1.id},
                {'type': 'input_text', 'text': (
                    'Analiza los indicadores financieros solicitados en el archivo de licitación'
                    'y compáralos con los indicadores entregados en el archivo base.'
                    'Para cada indicador, responde solo en formato JSON (sin texto adicional), con esta estructura exacta:\n\n'
                    '[\n'
                    '  {\n'
                    '    "descrip": "Nombre del indicador solicitado",\n'
                    '    "valor": valor_obtenido,\n'
                    '    "cumple": true/false\n'
                    '  },\n'
                    '  ...\n'
                    ']\n\n'
                    "No incluyas explicaciones ni texto adicional. Solo el arreglo JSON con los indicadores."
                )}
            ]
        }
    ]
)

In [21]:
# Se extrae y parsea la respuesta
salida = response.output_text.strip()

# Se valida que sea un JSON válido antes de parsear
try:
    lista_dicts = json.loads(salida)
    indicadores = [Indicadores(**item) for item in lista_dicts]
    for ind in indicadores:
        print(ind)
except Exception as e:
    print("Error al convertir la salida en objetos Indicadores:", e)
    print("Respuesta del modelo:\n", salida)


Error al convertir la salida en objetos Indicadores: Expecting value: line 1 column 1 (char 0)
Respuesta del modelo:
 ```json
[
  {
    "descrip": "Índice de Liquidez",
    "valor": 2.16,
    "cumple": true
  },
  {
    "descrip": "Índice de Endeudamiento",
    "valor": 0.47,
    "cumple": true
  },
  {
    "descrip": "Razón de Cobertura de Intereses",
    "valor": 19.83,
    "cumple": true
  },
  {
    "descrip": "Rentabilidad del Patrimonio",
    "valor": 0.26,
    "cumple": false
  },
  {
    "descrip": "Rentabilidad del Activo",
    "valor": 0.14,
    "cumple": false
  }
]
```
